In [ ]:
#| default_exp match.match_utils

# PSM Match functionalities

In [ ]:
#| export

import numpy as np
import numba
import pandas as pd
import tqdm

In [ ]:
#| export
@numba.njit
def match_centroid_mz(
    spec_mzs:np.ndarray, 
    query_mzs:np.ndarray, 
    spec_mz_tols:np.ndarray,
)->np.ndarray:
    """
    Matching query masses against sorted MS2/spec centroid masses, 
    only closest (minimal abs mass error) peaks are returned.
    
    Parameters
    ----------
    spec_mzs : np.ndarray
        MS2 or spec mz values, 1-D float array

    query_mzs : np.ndarray
        query mz values, n-D float array

    spec_mz_tols : np.ndarray
        Da tolerance array, same shape as spec_mzs

    Returns
    -------
    np.ndarray
        np.ndarray of int32, the shape is the same as query_mzs.
        -1 means no peaks are matched for the query mz
    """
    idxes = np.searchsorted(spec_mzs, query_mzs)
    ret_indices = np.empty_like(query_mzs, dtype=np.int32)
    # ret_indices[:] = -1
    for i,idx in np.ndenumerate(idxes):
        min_merr = abs(spec_mzs[idx-1]-query_mzs[i])
        min_idx = -1
        if min_merr <= spec_mz_tols[idx-1]:
            min_idx = idx-1
        if idx < len(spec_mzs):
            merr = abs(spec_mzs[idx]-query_mzs[i])
            if merr <= spec_mz_tols[idx] and merr < min_merr: 
                min_idx = idx
        ret_indices[i] = min_idx
    return ret_indices


@numba.njit
def match_profile_mz(
    spec_mzs:np.ndarray,
    query_mzs:np.ndarray,
    spec_mz_tols:np.ndarray,
    spec_intens:np.ndarray,
)->np.ndarray:
    """
    Matching query masses against sorted MS2/spec profile masses,
    only highest peaks are returned.

    Parameters
    ----------
    spec_mzs : np.ndarray
        MS2 or spec mz values, 1-D float array

    query_mzs : np.ndarray
        query mz values, n-D float array

    spec_mz_tols : np.ndarray
        Da tolerance array, same shape as spec_mzs

    Returns
    -------
    np.ndarray
        np.ndarray of int32, the shape is the same as query_mzs.
        -1 means no peaks are matched for the query mz
    """
    idxes = np.searchsorted(spec_mzs, query_mzs)
    ret_indices = np.empty_like(query_mzs, dtype=np.int32)
    for i,idx in np.ndenumerate(idxes):
        if idx == len(spec_mzs):
            idx = idx-1

        highest = 0
        highest_idx = -1
            
        for _idx in range(idx, -1, -1):
            if abs(spec_mzs[_idx]-query_mzs[i])>spec_mz_tols[_idx]:
                break
            if highest < spec_intens[_idx]:
                highest = spec_intens[_idx]
                highest_idx = _idx
        for _idx in range(idx+1, len(spec_mzs)):
            if abs(spec_mzs[_idx]-query_mzs[i])>spec_mz_tols[_idx]:
                break
            if highest < spec_intens[_idx]:
                highest = spec_intens[_idx]
                highest_idx = _idx
        ret_indices[i] = highest_idx
    return ret_indices

@numba.njit
def match_profile_first_last_mz(
    spec_mzs:np.ndarray,
    query_mzs:np.ndarray,
    spec_mz_tols:np.ndarray,
)->tuple:
    """
    Matching query masses against sorted MS2/spec profile masses,
    both first and last m/z values are returned.

    Parameters
    ----------
    spec_mzs : np.ndarray
        MS2 or spec mz values, 1-D float array

    query_mzs : np.ndarray
        query mz values, n-D float array

    spec_mz_tols : np.ndarray
        Da tolerance array, same shape as spec_mzs

    Returns
    -------
    tuple
        np.ndarray: matched first (left-most) indices, the shape is the same as query_mzs.
        -1 means no peaks are matched for the query mz.

        np.ndarray: matched last (right-most) indices, the shape is the same as query_mzs.
        -1 means no peaks are matched for the query mz.
    """
    idxes = np.searchsorted(spec_mzs, query_mzs)
    first_indices = np.empty_like(
        query_mzs, dtype=np.int32
    )
    last_indices = np.empty_like(
        query_mzs, dtype=np.int32
    )
    first_indices[:] = -1
    last_indices[:] = -1
    for i,idx in np.ndenumerate(idxes):
        if idx == len(spec_mzs):
            idx = idx-1
        for _idx in range(idx, -1, -1):
            if spec_mzs[_idx]<query_mzs[i]-spec_mz_tols[_idx]:
                break
            else:
                first_indices[i] = _idx
        for _idx in range(idx, len(spec_mzs)):
            if abs(spec_mzs[_idx]-query_mzs[i])>spec_mz_tols[_idx]:
                break
            else:
                last_indices[i] = _idx
    return first_indices, last_indices


In [ ]:
#| hide
spec_masses = np.array([100, 100.1, 199.9, 200.1, 300])
Da_tols = np.ones_like(spec_masses)*0.2
query_masses = np.array([100.0, 200, 300, 400])
first_indices, last_indices = match_profile_first_last_mz(
    spec_masses, query_masses, Da_tols,
)
assert np.all(first_indices==[ 0,  2,  4, -1])
assert np.all(last_indices==[ 1,  3,  4, -1])

In [ ]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
Da_tols = spec_masses*20*1e-6
query_masses = spec_masses
assert np.all(
    np.arange(10, dtype=np.int32)==
    match_centroid_mz(spec_masses, query_masses, Da_tols)
)

In [ ]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
Da_tols = spec_masses*20*1e-6
query_masses = spec_masses
spec_intens = np.ones_like(spec_masses)
assert np.all(
    np.arange(10, dtype=np.int32)==
    match_profile_mz(spec_masses, query_masses, Da_tols, spec_intens)
)

In [ ]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
Da_tols = spec_masses*20*1e-6
query_masses = spec_masses.copy()
query_masses[1]+=0.01 # -1
query_masses[3]+=0.003 # spec_masses[3]
query_masses[5]-=0.0099 # spec_masses[5]
query_masses[6]+=0.02 #-1
spec_masses[7] = spec_masses[8]-0.007 # matched[7] = -1
spec_masses[8] += 0.01 # matched[8] = 7 as 7 is closer
assert np.allclose(
    match_centroid_mz(spec_masses, query_masses, Da_tols),
    [0, -1,  2,  3,  4,  5, -1, -1,  7,  9]
)

In [ ]:
#| hide
#unittests
spec_masses = np.arange(10, dtype=float)*100
Da_tols = spec_masses*20*1e-6
query_masses = np.arange(20).reshape((10,2))*100
target = np.arange(20, dtype=np.int32)
target[10:] = -1
assert np.allclose(
    target.reshape((10,2)),
    match_centroid_mz(spec_masses, query_masses, Da_tols)
)